# RAG Cookbook

In this notebook, we show the useage of CAMEL `RetrievalModule` in both customized way and default auto way. We will also show how to combine `RetrievalModule` with `ChatAgent`, and further combine with `RolePlaying` by using `Function Calling`.

4 main parts included:
- Customized RAG
- Default RAG
- Single Agent with RAG
- Role-playing with RAG

## Load Data

Let's first load the CAMEL paper from https://arxiv.org/pdf/2303.17760.pdf. This will be our local example data.

In [1]:
import os
import requests

os.makedirs('local_data', exist_ok=True)

url = "https://arxiv.org/pdf/2303.17760.pdf"
response = requests.get(url)
with open('local_data/camel paper.pdf', 'wb') as file:
     file.write(response.content)

## 1. Customized RAG
In this section we will set our customized RAG pipeline.


Set embedding model, we will use `OpenAIEmbedding` as the embedding model, so we need to set the `OPENAI_API_KEY` in below.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "Your OpenAI Key"

Import and set the embedding instance:

In [3]:
from camel.embeddings import OpenAIEmbedding

embedding_instance = OpenAIEmbedding()

Import and set the vector storage instance:

In [4]:
from camel.storages.vectordb_storages import QdrantStorage

storage_instance = QdrantStorage(
    vector_dim=embedding_instance.get_output_dim(),
    collection="my first collection",
    path="storage_customized_run",
)

Import and set the retrieval instance:

In [5]:
from camel.retrievers.retrieval_module import RetrievalModule

retrieval_instance = RetrievalModule(embedding_model=embedding_instance)

We use integrated `Unstructured Moudle` to splite the content into small chunks, the content will be splited automacitlly with its `chunk_by_title` function, the max character for each chunk is 500 characters, which is a suitable length for `OpenAIEmbedding`. All the text in the chunks will be embed and stored to the vector storage instance, it will take some time, please wait..

In [6]:
retrieval_instance.embed_and_store_chunks(
    content_input_path="local_data/camel paper.pdf",
    vector_storage=storage_instance,
)

/Users/enrei/Library/Caches/pypoetry/virtualenvs/camel-ai-WbV-SHy3-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can retrieve information from the vector storage by giving a query. By default it will give you back the text content from top 1 chunk with highest Cosine similarity score, and the similarity score should be higher than 0.75 to ensure the retrieved content is relevant to the query. You can also change the `top_k` value and `similarity_threshold` value with your needs.

The returned string list includes:
- similarity score
- content path
- metadata
- text

In [7]:
retrieved_info = retrieval_instance.query_and_compile_results(
    query="What is CAMEL?", vector_storage=storage_instance, top_k=2
)

print(retrieved_info)

{'similarity score': '0.8321616118446469', 'content path': 'local_data/camel paper.pdf', 'metadata': {'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-01-13T23:44:25', 'page_number': 45}, 'text': 'CAMEL Data and Code License The intended purpose and licensing of CAMEL is solely for research use. The source code is licensed under Apache 2.0. The datasets are licensed under CC BY NC 4.0, which permits only non-commercial usage. It is advised that any models trained using the dataset should not be utilized for anything other than research purposes.\n\n45'}
{'similarity score': '0.8238950702386523', 'content path': 'local_data/camel paper.pdf', 'metadata': {'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-01-13T23:44:25', 'page_number': 67}, 'text': 'Table 7 presents the performance of LLaMA models fine-tuned on CAMEL role-play datasets from the manuscript (denoted CAMEL) and LLaMA models fine-tuned on CAMEL datasets in addition to Shar

Let's try an irrelevant query:

In [8]:
retrieved_info_irrevelant = retrieval_instance.query_and_compile_results(
    query="Compared with dumpling and rice, which should I take for dinner?",
    vector_storage=storage_instance,
    top_k=2,
)

print(retrieved_info_irrevelant)

No suitable information retrieved from
            local_data/camel paper.pdf with
            similarity_threshold = 0.75.


## 2. Default RAG
In this section we will run the auto RAG pipeline with default settings. It uses `OpenAIEmbedding` as default embedding model and `Qdrant` as default vector storage.

What you need to do is:
- Set content input paths, which can be local paths or remote urls
- Set local storage path or remote url and api key for Qdrant
- Give a query

The default RAG pipeline would create collections for given content input paths, the collection name will be set automaticlly based on the content input path name, if the collection exists, it will do the retrieval directly.

In [9]:
retrieval_instance_default = RetrievalModule()
retrieved_info = retrieval_instance_default.run_default_retrieval(
    content_input_paths=[
        "local_data/camel paper.pdf",  # example local path
        "https://www.camel-ai.org/",  # example remote url
    ],
    vector_storage_local_path="storage_default_run",
    query="What is CAMEL-AI",
)

print(retrieved_info)

Original Query:
{What is CAMEL-AI}
Retrieved Context:
{'similarity score': '0.8369126071945212', 'content path': 'local_data/camel paper.pdf', 'metadata': {'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-01-13T23:44:25', 'page_number': 7}, 'text': 'Section 3.2, to simulate assistant-user cooperation. For our analysis, we set our attention on AI Society setting. We also gathered conversational data, named CAMEL AI Society and CAMEL Code datasets and problem-solution pairs data named CAMEL Math and CAMEL Science and analyzed and evaluated their quality. Moreover, we will discuss potential extensions of our framework and highlight both the risks and opportunities that future AI society might present.'}
{'similarity score': '0.8380731206379989', 'content path': 'https://www.camel-ai.org/', 'metadata': {'emphasized_text_contents': ['Mission', 'CAMEL-AI.org', 'is an open-source community dedicated to the study of autonomous and communicative agents. We believe tha

## 3. Single Agent with RAG
In this section we will show how to combine the default `RetrievalModule` with one `ChatAgent`.

Let's set an agent function, in this function we can get the response by providing a query to this agent.

In [10]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.types import RoleType


def singe_agent(query: str) ->str :
    # Set agent role
    assistant_sys_msg = BaseMessage(
        role_name="Assistant",
        role_type=RoleType.ASSISTANT,
        meta_dict=None,
        content="You are a helpful assistant to answer question, I will give you the Original Query and Retrieved Context, answer the Original Query based on the Retrieved Context, if you can't answer the question just say I don't know.",
    )

    # Add default RAG
    retrieval_instance_default = RetrievalModule()
    retrieved_info = retrieval_instance_default.run_default_retrieval(
        content_input_paths=[
            "local_data/camel paper.pdf",  # example local path
            "https://www.camel-ai.org/",  # example remote url
        ],
        vector_storage_local_path="storage_default_run",
        query=query,
    )

    # Pass the retrieved infomation to agent
    user_msg = BaseMessage.make_user_message(role_name="User", content=retrieved_info)
    agent = ChatAgent(assistant_sys_msg)

    # Get response
    assistant_response = agent.step(user_msg)
    return assistant_response.msg.content


print(singe_agent("What is CAMEL-AI"))

CAMEL-AI is an open-source community dedicated to the study of autonomous and communicative agents. It provides, implements, and supports various types of agents, tasks, prompts, models, datasets, and simulated environments to facilitate research in this field. Additionally, it gathers conversational data and problem-solution pairs data named CAMEL Math and CAMEL Science for analysis and evaluation.


## 4. Role-playing with RAG
In this section we will show how to combine the default `RetrievalModule` with `RolePlaying` by applying `Function Calling`.


First, we need to set a retrieval function with well-written docstring for LLM to understand what this function is used for, the main code is the same with the default RAG section.

In [29]:
from typing import List
from camel.functions import OpenAIFunction
from camel.retrievers.retrieval_module import RetrievalModule


def local_retrieval(query: str) -> str:
    r"""Performs a default local retrieval for information. Given a query,
    this function will retrieve the information from the local vector storage,
    and return the retrieved information back. It is useful for information
    retrieval.

    Args:
        query (string): Question you want to be answered.

    Returns:
        str: Aggregated information retrieved in response to the query.

    Example:
        local_retrieval(query = "what is camel?")
    """
    retrieval_instance = RetrievalModule()
    retrieved_info = retrieval_instance.run_default_retrieval(
        content_input_paths=[
            "local_data/camel paper.pdf",  # example local path
            "https://www.camel-ai.org/",  # example remote url
        ],
        vector_storage_local_path="storage_default_run",
        query=query,
    )
    return retrieved_info


# add the function to OpenAIFunction list
RETRIEVAL_FUNCS: List[OpenAIFunction] = [
    OpenAIFunction(func)
    for func in [
        local_retrieval,
    ]
]

Run the role-playing with defined retrieval function:

In [31]:
from camel.utils import role_playing_with_function
from camel.functions import MATH_FUNCS  # import another function from camel

role_playing_with_function(
    task_prompt=(
        "What is AI society role-playing, how many assistant roles did camel generated, what's the value plus 100?"
    ),
    function_list=[*RETRIEVAL_FUNCS, *MATH_FUNCS],
)

AI Assistant sys message:
BaseMessage(role_name='Searcher', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={'task': "What is AI society role-playing, how many assistant roles did camel generated, what's the value plus 100?", 'assistant_role': 'Searcher', 'user_role': 'Professor'}, content="===== RULES OF ASSISTANT =====\nNever forget you are a Searcher and I am a Professor. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: What is AI society role-playing, how many assistant roles did camel generated, what's the value plus 100?. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately solves the requested instruction and explain your solutions.\nYou must decline my instruction honestly if you cannot perform the i